In [1]:
# load data
import pandas as pd
import ast
import os
import json
import shutil
from shutil import copyfile
from sklearn.model_selection import train_test_split
from PIL import Image

In [2]:
!pip install openpyxl

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p38/bin/python -m pip install --upgrade pip' command.


In [55]:
#download data
#!mkdir data_0731
#!aws s3 cp s3://jackie-test/bumingjueli/data0731/ ./data_0731/ --recursive
#!unzip ./data_0731/Sports.zip -d ./data_0731

# data prepare

In [8]:
#filter not 13 list
def get_feature_len(x):
    t = json.loads(x)
    return len(t)

def get_key_list(x):
    # get key dictionary
    t = json.loads(x)
    res = [i for i in list(t.values())]
    res = [list(i.keys())[0] for i in res]
    return res

def get_keys(df):
    lst = list(df['feature_dict'])
    myList = [x for j in lst for x in j]
    res = list(set(myList))
    #res_str = ','.join(res)
    return res
    
def map_feature(x,leng):
    t = json.loads(x)
    for i in range(leng):
        if str(i) in t.keys():
            continue
        else:
            t[str(i)] = ''
    return t

def get_res(x):
    try:
        a = ast.literal_eval(str(x))
        return a
    except:
        return {'res':'others'}
        
#map back labels
def get_label_txt():
    with open('./data/label.txt') as f:
        lines = f.readlines()
    
    keys =  [i.split('\t')[0] for i in lines]
    keys_update = [str(int(i)-1) for i in keys]
    res = [i.split('\t')[1][:-1] for i in lines ]
    dict_res = dict(zip(keys_update, res))
    return dict_res

def get_key_value(x,i):
#x = df['data'][59335]

    t = json.loads(x)

    res = [i for i in list(t.values())]
    keys = [list(i.keys())[0] for i in res]
    values = [list(i.values())[0] for i in res]
    dict_res = dict(zip(keys, values))
    if i in dict_res.keys():
        return dict_res[i]
    else:
        return 'other'


def test_path(x,category):
    root_path = os.path.join('/home/ec2-user/SageMaker/bumingjueli/img-cls/data/data_0731',category)
    img_name = os.path.join(root_path,str(x)+'.png')
    #print ('img_name',img_name)
    if os.path.exists(img_name):  
        # for local training
        #return img_name
        # for sagemaker only
        img = Image.open(img_name)
        if len(img.getbands())==3:
            return os.path.join('/opt/ml/input/data/training',str(x)+'.png')
        else:
            return 'none'
    else:
        return 'none'
def self_mkdir(folder):
    isExists = os.path.exists(folder)
    if not isExists:
        os.makedirs(folder)
        print('path of %s is build' % (folder))

def copy_files(df,category,output_dir):
    for i in df['md5_url']:
        copyfile(os.path.join('./data_0731',category,i+'.png'),os.path.join(output_dir,i+'.png'))
        
        
def get_data(path,category,output_dir):
    df = pd.read_excel(path,engine="openpyxl")
    df = df[df['creg']==category]
    #df['feature_len'] = df['data'].map(lambda x: get_feature_len(x))
    #leng = max(df['feature_len'])
    df['feature_dict'] = df['data'].map(lambda x: get_key_list(x))
    res_keys = get_keys(df)
    print ("<<< predict for keys: ", ','.join(res_keys))
    
    for i in res_keys:
        df[i] = df['data'].map(lambda x: get_key_value(x,i))
    
    #repath
    df['image_path'] = df['md5_url'].map(lambda x: test_path(x,category))
    df = df[df['image_path']!='none']
    
    #make dir if not exist
    self_mkdir(output_dir)
    #save data
    df[res_keys].to_csv(os.path.join(output_dir, 'total.csv'),index=False)
    
    #sample
    #df = df.head(20)
    #copy images
    copy_files(df,category,output_dir)
    
    train, test = train_test_split(df,test_size=0.25,random_state=0)
    train.to_csv(os.path.join(output_dir, 'train.csv'))
    test.to_csv(os.path.join(output_dir, 'test.csv'))
    print ("train size {}, test size{}".format(train.shape,test.shape))
    
    return df

category = 'Women-Sweatshirts'
output_dir = os.path.join("./train_data",category)
df = get_data('./data_0731/shein_info.xlsx',category=category,output_dir = output_dir)

<<< predict for keys:  Pockets,Lining,Sheer,Pattern Type,Neckline,Chest pad,Sleeve Type,Length,Fabric,Support,Fit Type,Belt,Waterproof,Quantity,Details,Number of Pieces,Filling,Sleeve Length,Care Instructions,Warm Lined,Panty Type,Color,Features,Bra Type,Activity,Type,Body,Composition,Material,Waist Line
train size (4975, 39), test size(1659, 39)


# train

In [ ]:
import sagemaker as sage
from sagemaker import get_execution_role
from sagemaker.pytorch import PyTorch

sess = sage.Session()

WORK_DIRECTORY = output_dir

# S3 prefix
prefix = "bmjl-train-"+category

role = get_execution_role()

data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)

In [ ]:
data_location

In [ ]:
hyperparameters = {
    "epoch":50,
    "batch_size":4,
    "num_workers":8,
    'val_epoch':1,
    'save_epoch':10,
}

In [ ]:
entry_point = 'train_general_sagemaker.py'
source_dir = './code'
git_config = None
role = get_execution_role()
framework_version = '1.7.1'
py_version='py36'
instance_type='ml.p3.2xlarge'
#instance_type='local_gpu'
instance_count=1
volume_size=50

In [ ]:
estimator = PyTorch(
    entry_point = entry_point,
    source_dir = source_dir,
    git_config = git_config,
    role = role,
    debugger_hook_config=False,
    hyperparameters = hyperparameters,
    framework_version = framework_version, 
    py_version = py_version,
    instance_type = instance_type,
    instance_count = instance_count,
    base_job_name = base_job_name,
    volume_size=volume_size
)

In [ ]:
response = estimator.fit(data_location)